# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_df = pd.read_csv("cities.csv")
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hilo,90,US,1633023610,94,19.7297,-155.0900,73.80,6.91
1,Bredasdorp,64,ZA,1633023856,77,-34.5322,20.0403,60.12,12.66
2,Hamilton,75,US,1633023820,48,39.1834,-84.5333,79.23,1.01
3,Caravelas,25,BR,1633023789,60,-17.7125,-39.2481,82.02,10.98
4,Ostrovnoy,75,RU,1633023891,87,68.0531,39.5131,43.07,3.06
...,...,...,...,...,...,...,...,...,...
572,Merritt Island,20,US,1633024102,60,28.5392,-80.6720,90.10,9.22
573,Mujiayingzi,2,CN,1633024102,62,42.1167,118.7833,49.30,2.55
574,Ballina,40,AU,1633023830,88,-28.8667,153.5667,64.36,10.36
575,Bubaque,23,GW,1633023886,75,11.2833,-15.8333,82.89,10.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [25]:
# Plot Heatmap
map = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heatlayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
# Add layer
map.add_layer(heatlayer)
# Display map
map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                         (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
104,San Quintín,0,MX,1633023929,64,30.4833,-115.9500,71.96,7.07
198,El Dorado,0,US,1633023962,20,38.7666,-120.5177,76.12,4.29
220,Shache,0,CN,1633023971,15,38.4167,77.2406,71.02,5.21
282,Nanjing,0,CN,1633023702,75,32.0617,118.7778,70.77,3.06
304,Upington,0,ZA,1633024002,17,-28.4478,21.2561,75.58,5.75
328,Kiryat Gat,0,IL,1633024011,100,31.6100,34.7642,77.68,5.73
377,San Bernardo,0,CL,1633024029,20,-33.6000,-70.7167,70.07,6.96
404,Guerrero Negro,0,MX,1633024039,57,27.9769,-114.0611,72.48,9.24
432,Buin,0,CL,1633024049,21,-33.7333,-70.7500,71.06,8.84
533,Huesca,0,ES,1633024087,47,42.1362,-0.4087,70.95,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
104,San Quintín,MX,30.4833,-115.9500,
198,El Dorado,US,38.7666,-120.5177,
220,Shache,CN,38.4167,77.2406,
282,Nanjing,CN,32.0617,118.7778,
304,Upington,ZA,-28.4478,21.2561,
328,Kiryat Gat,IL,31.6100,34.7642,
377,San Bernardo,CL,-33.6000,-70.7167,
404,Guerrero Negro,MX,27.9769,-114.0611,
432,Buin,CL,-33.7333,-70.7500,
533,Huesca,ES,42.1362,-0.4087,


In [33]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    cityname = row["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {cityname}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {cityname} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing data... skipping.")


Retrieving Results for Index 104: San Quintín.
Closest hotel in San Quintín is Don Eddie's Landing.
Retrieving Results for Index 198: El Dorado.
Closest hotel in El Dorado is Sly Park Resort.
Retrieving Results for Index 220: Shache.
Closest hotel in Shache is Shache Wanghou Hotel.
Retrieving Results for Index 282: Nanjing.
Closest hotel in Nanjing is InterContinental Hotel Nanjing.
Retrieving Results for Index 304: Upington.
Closest hotel in Upington is Desert Palace Hotel & Casino Resort.
Retrieving Results for Index 328: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
Retrieving Results for Index 377: San Bernardo.
Missing data... skipping.
Retrieving Results for Index 404: Guerrero Negro.
Closest hotel in Guerrero Negro is The Halfway Inn.
Retrieving Results for Index 432: Buin.
Closest hotel in Buin is Motel INSIDE.
Retrieving Results for Index 533: Huesca.
Closest hotel in Huesca is abba Huesca Hotel.


In [29]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
104,San Quintín,MX,30.4833,-115.9500,Don Eddie's Landing
198,El Dorado,US,38.7666,-120.5177,Sly Park Resort
220,Shache,CN,38.4167,77.2406,Shache Wanghou Hotel
282,Nanjing,CN,32.0617,118.7778,InterContinental Hotel Nanjing
304,Upington,ZA,-28.4478,21.2561,Desert Palace Hotel & Casino Resort
328,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
377,San Bernardo,CL,-33.6000,-70.7167,
404,Guerrero Negro,MX,27.9769,-114.0611,The Halfway Inn
432,Buin,CL,-33.7333,-70.7500,Motel INSIDE
533,Huesca,ES,42.1362,-0.4087,abba Huesca Hotel


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer and info box content ontop of heat map
hotel = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
map.add_layer(hotel)

# Display Map
map

Figure(layout=FigureLayout(height='420px'))